In [50]:
import pandas as pd
import praw

In [51]:
qbs_and_clusters = pd.read_csv("https://raw.githubusercontent.com/jake-lukasik/NFL-Player-Clustering/refs/heads/main/filtered-data/QBS-and-Comments.csv").drop(columns=["Unnamed: 0"])
display(qbs_and_clusters)

,Player,Team,Cluster,reddit_comments
0,Joe Burrow,CIN,2,['Joe Burrow bought those so they could attack...
1,Jared Goff,DET,2,"['Jared Goff is elite at play action', 'Jared ..."
2,Baker Mayfield,TAM,2,"[""If you don't like Baker Mayfield, fuck you.""..."
3,Geno Smith,SEA,0,"['GENO SMITH BENDS THE KNEE!!!', 'Geno Smith a..."
4,Sam Darnold,MIN,2,['Surely Sam Darnold succeeding Geno Smith wil...
5,Lamar Jackson,BAL,2,['I feel so fucking lucky to be a Ravens and L...
6,Patrick Mahomes,KAN,0,"['Super Bowl Mvp Patrick Mahomes', '""Chiefs QB..."
7,Aaron Rodgers,NYJ,0,['It must be weird to be spoken to with your f...
8,Justin Herbert,LAC,0,"['Maybe I’m Justin Herbert 🤔', 'This just in…J..."
9,Brock Purdy,SFO,3,['Yeah really depends on how you view it lol I...


In [52]:
# iterate over 0-3 and isolate each cluster into their own df
cluster_dfs = [
    qbs_and_clusters[qbs_and_clusters["Cluster"] == i]
    for i in range(4)
]
display(cluster_dfs[0]) # access each individual cluster like this

,Player,Team,Cluster,reddit_comments
3,Geno Smith,SEA,0,"['GENO SMITH BENDS THE KNEE!!!', 'Geno Smith a..."
6,Patrick Mahomes,KAN,0,"['Super Bowl Mvp Patrick Mahomes', '""Chiefs QB..."
7,Aaron Rodgers,NYJ,0,['It must be weird to be spoken to with your f...
8,Justin Herbert,LAC,0,"['Maybe I’m Justin Herbert 🤔', 'This just in…J..."
10,Kyler Murray,ARI,0,"['* **No. 1 overall — Oklahoma’s Kyler Murray,..."
11,Bo Nix,DEN,0,['that went about 66 air yards and i thought b...
12,Matthew Stafford,LAR,0,['That’s super bowl champion Matthew Stafford'...
14,C.J. Stroud,HOU,0,['I\'ve seen people claim that it\'s been disp...
15,Jayden Daniels,WAS,0,['Does Jayden Daniels not care about winning? ...
16,Caleb Williams,CHI,0,"['Caleb Williams had plenty of flaws, although..."
